# Colab 3: Implement Bloom Filter using Spark

## Setup

Let's setup Spark on your Colab environment.  Run the cell below!

---



In [132]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u282-b08-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


In [133]:
#Import libraries
import hashlib
import math
import pyspark
from pyspark.sql import *
import pyspark.sql.functions as f
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext

# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
try:
  sc.stop()
  sc = pyspark.SparkContext(conf=conf)
except:
  sc = pyspark.SparkContext(conf=conf)

spark = SparkSession.builder.getOrCreate()

### Data Loading

From the NLTK (Natural Language ToolKit) library, we import a large list of English dictionary words, commonly used by the spell-checking programs.

In [134]:
import nltk
nltk.download('words')

from nltk.corpus import words
word_list = words.words()

# Convert word_list to a Spark RDD
word_rdd = spark.sparkContext.parallelize(word_list)


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


## Determine the size of the hash table
In order to build a Bloom Filter, we need to determine the size of the hash table which is calcuated based on the number of words in the dictionary and the number of hash funcitons we plan to use. The following `get_digits` function returns the number of digits we need to take out of each hash function to get the indices.

In [135]:
def get_digits(n, hash_num):

    # m gives how many indices we need
    m = hash_num * n / (math.log(2))
    # Number of digits we need to take out of each hash function to get the indices
    num_of_digits = len(str(int(m)))

    return num_of_digits

In [136]:
# get the total number of words in the dictionary
wordCount = word_rdd.count()
print(wordCount)
# set the number of hash functions to 4
NUM_HASHFUNCTIONS = 4

num_of_digits = get_digits(wordCount, NUM_HASHFUNCTIONS)
print(num_of_digits)
print(math.pow(10,num_of_digits))

236736
7
10000000.0


## Get Hash Values
The `hash_values` funciton takes a key and the `num_of_digits` as inputs, and returns 4 indices using 4 secure hash functions from Python [hashlib](https://docs.python.org/3/library/hashlib.html) library.

In [137]:
def hash_values(item, num_of_digits):
  # Get the index value to check for the four hash functions we are using
    index_md5    = int(hashlib.md5(item.encode("utf-8")).hexdigest(),16) % (10 ** num_of_digits)
    index_sha1   = int(hashlib.sha1(item.encode("utf-8")).hexdigest(),16) % (10 ** num_of_digits)
    index_sha512 = int(hashlib.sha512(item.encode("utf-8")).hexdigest(),16) % (10 ** num_of_digits)
    index_sha384 = int(hashlib.sha384(item.encode("utf-8")).hexdigest(),16) % (10 ** num_of_digits)

    return index_md5, index_sha1, index_sha512, index_sha384

##Build Bloom Filter
Now we are ready to use the words from the dictionary to build the Bloom Filter. Note that in the original Bloom Filter algorithm introcuded in class, we used a bit array and set the indices to 1 to mark the existance of a valid element from the dictionary. Now we want to implemment it in a different way. Rather than return a bit arrary, we want to return all the valid indices instead. For example, assume that by using the orignial Bloom FIlter algorithm, we get a bit array (0, 0, 1, 0, 1, 1, 1, 0, 1, 1). Now we would like to return the indices of 1s, i.e., (2, 4, 5, 6, 8, 9).

**You need to fill in the code in the function below.**

In [138]:
def build_bloom_filter(in_data, num_of_digits):
    
    # Split the input file into multiple lines
    in_split = in_data.flatMap(lambda x: x.split())
    
    # could only get it to work after converting it into a RDD from a PipelinedRDD 
    # I don't know what that didn't work otherwise
    rdd = spark.sparkContext.parallelize(in_split.collect())


    # Apply four different hash functions to create the Bloom filter
    # Each one of these RDDs has the indice that will be set to 1 to mark the existance of the input elements

    in_md5 =  rdd.map(lambda x: hash_values(x, num_of_digits)[0])
    in_sha1 =  rdd.map(lambda x: hash_values(x, num_of_digits)[1])
    in_sha512 =  rdd.map(lambda x: hash_values(x, num_of_digits)[2])
    in_sha384 =  rdd.map(lambda x: hash_values(x, num_of_digits)[3])

    
    # Union all sets of indices, so we know the full collection of indices that have to be set to 1
    in_allhashes = sc.union([in_md5,in_sha1,in_sha512,in_sha384])

    # Remove duplicates
    in_distinctIndex = in_allhashes.distinct()

    # how to test to make sure it works


    # non_unique_nums = spark.sparkContext.parallelize(in_allhashes.collect())
    # print(non_unique_nums.takeOrdered(10, key=lambda x: x))
    # nums = spark.sparkContext.parallelize(in_distinctIndex.collect())
    # print(nums.takeOrdered(10, key=lambda x: x))


    # Return all the indices of value 1
    return in_distinctIndex



In [139]:
# call build_bloom_filter() function to get all the valid indices
valid_indices = build_bloom_filter(word_rdd, num_of_digits)



## Test set membership using Bloom Filter

For each word in the test set, we need to determine is it is a valid word in the dictionary. We need to apply the four hash functions introcuded earlier, and determine if all four hash incides are valid indices.

**You need to fill in the code in the function below.**

In [140]:
def bloom_filter(valid_indices, num_of_digits, test):

    for item in test:
        #print(type(valid_indices))
        # Checks if an item exists in a set of indices for a bloom filter
        index_md5, index_sha1, index_sha512, index_sha384 = hash_values(item, num_of_digits)

        # Use filter() function to determine if each of the indices is a member of valid_indices
        indexes = [index_md5, index_sha1, index_sha512, index_sha384]

      # I think this is a cleaner way of doing it. 
        # this rdd is the index of the proper hash values. 
        # If there are 4 integers (eg the word has 4 valid indexes) then
        # either it is in the set or there are 4 false positives on each of the hash functions. 
        # since this method makes certain the that members of the set pass
        # as long as there are 4 elements in the set then is possibly passes. 
        # not equal to 4 is a certain fail. 

        indexes_found_in_valid = valid_indices.filter(lambda x: x in indexes)

        all_found = (indexes_found_in_valid.count() == 4)

        # Print message with results depending on the value of all_found
        if all_found:
            print("'", item, "' is possibly in the dictionary.")
        else:
            print("'", item, "' is not in the dictionary for sure.")



bloom_filter(valid_indices, num_of_digits, test_set)


' I ' is possibly in the dictionary.
' am ' is possibly in the dictionary.
' nnot ' is not in the dictionary for sure.
' good ' is possibly in the dictionary.
' at ' is possibly in the dictionary.
' speling ' is not in the dictionary for sure.
' Pleasee ' is not in the dictionary for sure.
' dont ' is possibly in the dictionary.
' laught ' is not in the dictionary for sure.
' at ' is possibly in the dictionary.
' me ' is possibly in the dictionary.


## Get Test Data
Now we authenticate a Google Drive client to download files. Please follow the instruction to enter the authoriztion code.


In [141]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [142]:
id='1QhWtO03km0oP8EBgEco2zvuLok7K5riI'
downloaded = drive.CreateFile({'id': id}) 
downloaded.GetContentFile('test.txt') 

In [143]:
test_set = [ word for word in map(lambda x: x.strip(), open("test.txt").readlines()) ]

test_set


['I',
 'am',
 'nnot',
 'good',
 'at',
 'speling',
 'Pleasee',
 'dont',
 'laught',
 'at',
 'me']

##Apply Bloom Filter to the test data

In [144]:
bloom_filter(valid_indices, num_of_digits, test_set)

' I ' is possibly in the dictionary.
' am ' is possibly in the dictionary.
' nnot ' is not in the dictionary for sure.
' good ' is possibly in the dictionary.
' at ' is possibly in the dictionary.
' speling ' is not in the dictionary for sure.
' Pleasee ' is not in the dictionary for sure.
' dont ' is possibly in the dictionary.
' laught ' is not in the dictionary for sure.
' at ' is possibly in the dictionary.
' me ' is possibly in the dictionary.
